<a href="https://colab.research.google.com/github/NicholasBaraghini/Ball-and-Beam-system-Optimal-Control/blob/main/OPTCON_Grp21_Ball_and_Beam_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**OPTIMAL CONTROL EXAM PROJECT**

 Group 21:
   - Baraghini Nicholas 0000984041 
   - Curto Fabio
   - Iadarola Federico

In [2]:
# Libraries
import numpy as np
import matplotlib

In [27]:
# USEFULL FUNCTIONs

def dot3(a,B,c):
  # INPUTs: 
  #   a : row vector 1xN
  #   B : matrix NxN
  #   c : column vector Nx1

  return np.matmul(np.matmul(a,B),c) # Returns the matrix product a*B*c 

**SYSTEM DYNAMICS DYNAMICS** (Ball and Beam)

In [3]:
# SYSTEM DYNAMIC FUNCTION 

def BB_Dynamics(xx, uu, p_tens, params):
  # INPUTs:
  #   - xx  : system state at current time t 
  #   - uu  : input at current time t
  #   - p_tens: tensor product term
  #   - params: list of parameters

  # PARAMETERs EXTRACTION:
  dt = params['dt']; # Step size - Forward Euler method 
  gg = params['gg']; # gravitational acceleration [m/s^2]
  mm = params['mm']; # ball mass [kg]
  rr = params['rr']; # ball radius [m]
  ii = params['ii']; # ball inertia [kg*m^2]
  II = params['II']; # beam inertia [kg*m^2]
  LL = params['LL']; # beam lenght [m]
  
  # USEFUL VARIABLEs
  nx = 4; # nmumber of states 
  nu = 1; # number of inputs


  # SYSTEM DYNAMICS
  xx_next = np.zeros((nx,1)); # initialization
  
  xx_dot = np.array([[xx[1], 
                     ( mm*xx[0]*(xx[3]**2) - mm*gg*np.sin(xx[2]) )/( mm + II/(rr**2) ),
                     xx[3],
                     -( 2*mm*xx[0]*xx[1]*xx[3] + mm*xx[0]*np.cos(xx[2]) - uu )/d22  ]]);

  xx_next[0] = xx[0] + xx_dot[0]*dt;
  xx_next[1] = xx[1] + xx_dot[1]*dt;
  xx_next[2] = xx[2] + xx_dot[2]*dt;
  xx_next[3] = xx[3] + xx_dot[3]*dt;


  # GRADIENTs

  # useful notations
  d1  = ( mm + ii/(rr**2) )**(-1);
  d2  = ( II + mm*(xx[0]**2))**(-1);
  d22 = ( II + mm*(xx[0]**2));
  

  fx1_4_num = ( -( 2*mm*xx[1]*xx[3] + mm*gg*np.cos(xx[2]) )*d22 
                +( 2*mm*xx[0]*xx[1]*xx[3] + mm*gg*xx[0]*np.cos(xx[2]) - uu )*(2*mm*xx[0]) );
  fx1_4_den = d22**(2); 

  # partial derivative w.r.t. xx[1]:
  fx1 = np.array([[1, 
                  dt * ( mm*(xx[3]**2)*d1 ),
                  0,
                  dt * (fx1_4_num / fx1_4_den) ]]); 

  # partial derivative w.r.t. xx[2]:
  fx2 = np.array([[dt,
                  1,
                  0,
                  dt * ( -2*mm*xx[0]*xx[3]*d2 )]]);
      
  # partial derivative w.r.t. xx[3]:
  fx3 = np.array([[0,
                  dt * ( -mm*gg*np.cos(xx[2])*d1 ),
                  1,
                  dt * ( mm*gg*xx[0]*np.sin(xx[2])*d2 ) ]]);
  
  # partial derivative w.r.t. xx[4]:
  fx4 = np.array([[0,
                  dt * ( 2*xx[0]*xx[3]*d1 ),
                  dt,
                  1 - dt * ( 2*mm*xx[0]*xx[1]*d2 )]]);
  
  # Jacobian of the system dynamics:
  fx = np.concatenate((fx1.T,fx2.T,fx3.T,fx4.T), axis=1);

  # partial derivative w.r.t. the input:
  fu = np.array([[0,
                  0,
                  0,
                  dt*d2]]);


  # SECOND ORDER GRADIENTs
  pfxx = np.zeros((nx,nx));
  pfux = np.zeros((nu,nx));
  pfuu = np.zeros((nu,nu));
  
  # useful notations
  fx1x1_4_num   = (2*mm*( 2*mm*xx[0]*xx[1]*xx[3] + mm*gg*np.cos(xx[3]) - uu )*fx1_4_den)-(fx1_4_num *(4*d22*mm*xx[0]));
  fx1x1_4_den = d22**(4);
  

  # 1st row of the second derivative matrix nx*nx
  pfxx[0,0] = pp[3] * (( -2*mm*xx[3]*d22 + 4*xx[3]*(mm*xx[0])**2 ) / fx1_4_den) * dt;
  pfxx[0,1] = pp[3] * (( mm*gg*np.sin(xx[2])*d22 - 2*gg*np.sin(xx[2])*(mm*xx[0])**2)  / fx1_4_den) * dt;
  pfxx[0,2] = pp[3] * (( -2*mm*xx[1]*d22 + 4*xx[1]*(mm*xx[0])**2) / fx1_4_den) * dt;
  pfxx[0,3] = ( pp[1] * d1* (2*mm*xx[3]) * dt +
                pp[3] * (fx1x1_4_num  / fx1x1_4_den) * dt );
  
  # 2nd row of the second derivative matrix nx*nx
  pfxx[1,0] = pp[3] * (-2*mm*xx[3]*d2) * dt;
  pfxx[1,1] = 0;
  pfxx[1,2] = 0;
  pfxx[1,3] = pp[3] * (-2*mm*xx[0]*d2) * dt;

  # 3rd row of the second derivative matrix nx*nx
  pfxx[2,0] = pp[3] * (mm*gg*np.cos(xx[2])*d2) * dt;
  pfxx[2,1] = 0;
  pfxx[2,2] =( pp[1] * (mm*gg*np.sin(xx[2])*d1) * dt +
               pp[3] * (-mm*gg*xx[0]*np.sin(xx[2])*d2) * dt );
  pfxx[2,3] = 0;

  # 4th row of the second derivative matrix nx*nx
  pfxx[3,0] =( pp[1] * (2*mm*xx[3]*d1) * dt +
               pp[3] * (-2*mm*xx[1]*d2) * dt );
  pfxx[3,1] = pp[3] * (-2*mm*xx[0]*d2) * dt;
  pfxx[3,2] = 0;
  pfxx[3,3] = pp[1] * (2*mm*xx[0]*d1) * dt;

  # pfuu has all null elements

  # pfux has only one non-null element
  pfux[3,1] = pp[3] * (2*mm*x[0]*(d2**2)) * dt;


  # OUTPUTs: (the fucnction returns an output dictionary with the follows entries)
  #   - xx_next : system state at state (t+1)
  #   - fx     : gradient of the system dynamics w.r.t. the system state at time t
  #   - fu     : gradient of the system dynamics w.r.t. the input at time t
  #   - pfxx   : tensor product within the dynamics function seconnd order derivative w.r.t. the state, given vector pp
  #   - pfux   : tensor product within the dynamics function seconnd order derivative w.r.t. input and state, given vector pp 
  #   - pfuu   : tensor product within the dynamics function seconnd order derivative w.r.t. input, given vector pp 
  out = {
         'xx_next':xx_next,
         'fx':fx,
         'fu':fu,
         'pfxx':pfxx,
         'pfux':pfux,
         'pfuu':pfuu 
        };

  return out;

**COST FUNCTION IMPLEMENTATION**

In [4]:
# STAGE COST FUNCTION

def Stage_Cost(xx, uu, xx_ref, uu_ref, params):
  # INPUTs:
  #   - xx  : system state at current time t 
  #   - uu  : input at current time t
  #   - xx_ref: referance state at time t
  #   - uu_ref: referance input at time t
  #   - params: list of parameters

  QQ = params['QQ'];
  RR = params['QQ_T'];

  nx = np.shape(xx_ref)[0] # state vector dymension
  nu = np.shape(uu_ref)[0] # input vector dymension

  state_err = (xx - xx_ref);
  input_err = (uu - uu_ref);

  L_t = dot3(state_err.T, QQ, state_err) + dot3(input_err.T, RR, input_err); # cost function evaluated at time t

  # GRADIENTs
  DLx = 2*np.matmul(QQ,xx) - 2*np.matmul(QQ,xx_ref);
  DLu = 2*np.matmul(RR,uu) - 2*np.matmul(RR,uu_ref); 

  # 2nd order GRADIENTs
  DLxx = 2*QQ; 
  DLuu = 2*RR; 
  DLux = np.array(np.zeros((nu,nx)));

  # OUTPUTs: (the fucnction returns an output dictionary with the follows entries)
  #   - cost_t : cost evaluated at time t
  #   - Dlx    : gradient of the cost w.r.t. the system state at time t
  #   - Dlu    : gradient of the cost dynamics w.r.t. the input at time t
  #   - DLxx   : hessian w.r.t. the system state at time t
  #   - DLux   : hessian w.r.t. ux 
  #   - Dluu   : hessian w.r.t. the input at time t
  out = {
         'cost_t':L_t,
         'DLx':Dlx,
         'DLu':DLu,
         'DLxx':Dlxx,
         'DLux':Dlux,
         'Dluu':DLuu 
        };

  return out;


In [5]:
# TERMINAL COST FUNCTION

def Terminal_Cost(xx_T, xx_T_ref, params):
  # INPUTs:
  #   - xx_T  : system state at current final time T 
  #   - xx_T_ref: referance state at final time T
  #   - params: list of parameters

  QQ_T = params['QQ_T'];

  nx = np.shape(xx_T_ref)[0] # number of rows of xx_T_ref

  state_err = (xx_T - xx_T_ref);

  L_T = dot3(state_err.T, QQ, state_err); # cost function evaluated at final time T

  # GRADIENTs
  DLx = 2*np.matmul(QQ_T,xx_T) - 2*np.matmul(QQ_T,xx_T_ref);

  # 2nd order GRADIENTs
  DLxx = 2*QQ_T; 

  # OUTPUTs: (the fucnction returns an output dictionary with the follows entries)
  #   - cost_T : cost evaluated at final time T
  #   - Dlx    : gradient of the cost w.r.t. the system state at final time T
  #   - DLxx   : hessian w.r.t. the system state at final time T

  out = {
         'cost_T':L_T,
         'DLx':Dlx,
         'DLxx':Dlxx,
        };

  return out;

**DDP Components**\
Function that computhe the element K, P, p, σ and the descent direction from the DDP algorithm. 

In [46]:
# DDP Algorithm Components evaluated at k-th iteration
def DDP_comp_t_k(kk, xx, uu, xx_ref, uu_ref, params):
  # INPUTs:
  #   - kk     : iteration of evaluation
  #   - xx     : systema state tensor
  #   - uu     : input tensor
  #   - xx_ref : system state reference matrix
  #   - uu_ref : input reference matrx
  #   - tt     : time of evaluation
  #   - TT     : final step T
  #   - params : parameters dictionary

  nx = np.shape(xx_ref)[0] # state vector dymension
  nu = np.shape(uu_ref)[0] # input vector dymension

  # Initializations
  KK = np.zeros((nu,nx, TT));
  SS = np.zeros((nu, TT)); # sigma
  pp = np.zeros((nn, TT));
  PP = np.zeros((nn,nn, TT));

  for tt in range(TT-1, 0, -1):
    # System dymanics ar time t k-th iteration
    dyn = BB_Dynamics(xx[:,tt,kk], uu[:,tt,kk], pp[:,tt+1], params); 
    # stage cost at time t k-th iteration
    stC = Stage_Cost(xx[:,tt,kk], uu[:,tt,kk], xx_ref[:,tt], uu_ref[:,tt], params); 
    # terminal cost at time t k-th iteration
    trC = Terminal_Cost(xx[:,tt,kk], xx_ref[:,tt], params); 

    
    # Gain Computation
    KS_dir_term = stC['luu'] + dot3( dyn['fu'].T, PP[:,:,tt+1], dyn['fu'] ) + dyn['pfuu'];
    KS_inv_term = np.linalg.inv(KS_dir_term); # inverse factor of the DDP gain formula
    KK_dir_term = stC['lux'] + dot3( dyn['fu'].T, PP[:,:,tt+1], dyn['fx'] ) + dyn['pfux']; # second factor of the DDP gain formula

    KK = -np.matmul(KS_inv_term, KS_dir_term);

    # Sigma Computation
    SS_dir_term = stC['lu'] + np.matmul( dyn['fu'], pp[:,tt+1] ); # second factor of the DDP sigma formula

    SS = -np.matmul(KS_inv_term, SS_dir_term);

    # PP update 
    PP_1_term = stC['lxx'] + dot3(dyn['fx'].T,PP[:,:,tt+1],dyn['fx']) + dyn['pfxx']; # PP first term (DDP formula)
    PP_2_term = dot3( np.linalg.inv( KK[:,:,tt]).T, KS_dir_term, KK[:,:,tt]) # PP second term (DDP formula)
    
    PP[:,:,tt] = PP_1_term - PP_2_term;
    
    PP[:,:,TT] = trC['DLxx'];

    # pp update
    pp_1_term = stC['lx'] + np.matmul(dyn['fx'].T, pp[:,tt+1]); # PP first term (DDP formula)
    pp_2_term = dot3( np.linalg.inv( KK[:,:,tt]).T, KS_dir_term, SS[:,tt]) # PP second term (DDP formula)

    pp[:,tt] = pp_1_term - pp_2_term;

    pp[:,TT] = trC['DLx'];

    # Descent Direction Computation
    descent[kk] = descent[kk] - np.matmul(SS[:,tt].T, SS[:,tt]);

  # OUTPUTs:
  #   - KK      :
  #   - Sigma   :
  #   - PP      :
  #   - pp      :
  #   - descent :

  out = {
         'KK'     :KK,
         'Sigma'  :SS,
         'PP'     :PP,
         'pp'     :pp,
         'descent':descent
        };

  return out;

In [41]:
def op(a,d) :
  out = {'+':a+d,
         '-':a-d,
         '--':d-a,
         '*':a*d,
         '/':a/d,
         '//':d/a}
  return out;

b = op(5,10)
b['+']


15

**ARMIJO's ALGORITHM**

In [47]:
# ARMIJO's Function

def Armijo(kk, xx_init, xx_ref, uu_ref, nx, nu, TT, cost_kk, cc, beta, Sigma, KK, pp, params):
  # INPUTs:
  #   - kk       : actual iteration
  #   - xx_init  : system initial state at at time t=0 
  #   - xx_ref   : referance state vector
  #   - uu_ref   : reference input vector 
  #   - TT       : final step T 
  #   - cost_kk  : cost at k-th iteration
  #   - cc       : 
  #   - beta     : step size of the Armijo's algorithm
  #   - Sigma    : Control Affine Element from the DDP algorithm
  #   - KK       : Feedback Gain Matrix frm the DDP algorithm
  #   - pp       : vector p from the DDP algorithm 
  #   - params   : parameter dictionary

  nx = np.shape(xx_ref)[0] # state vector dymension
  nu = np.shape(uu_ref)[0] # input vector dymension

  gammas = np.array([[1]])     # initial step size
  armijo_cost=np.array([[]])   #cost for armijo function, update each iteration
  
  #temporary variable initialization
  xx_temp = np.zeros((nx,TT))
  uu_temp = np.zeros((nu,TT))
  
  # ARMIJO's LOOP
  while True:
    xx_temp[:,0] = xx_init
    cost_temp = 0

    for tt in range(0, TT):
      # temporary input control computation
      uu_temp[:,tt] = uu[:,tt,kk] + gammas[-1]*sigma[:,tt] + np.matmul(KK[:,:,tt], (xx_temp[:,tt]-xx[:,tt,kk])) #!! KK e sigma missing 
      # temporary system dynamics computation
      xx_temp[:,tt+1] = BB_Dynamics(xx_temp[:,tt], uu_temp[:,tt], pp[:,tt+1] , params )['xx_next']
      # stage cost computation
      cost_dummy = Stage_Cost(xx_temp[:,tt], uu_temp[:,tt], xx_ref[:,tt], uu_ref[:,tt], params)['cost_t']

      # cost sum at for each stage cost in time [0,T-1]
      cost_temp += cost_dummy
    
    # cost sum at for each stage cost in time [0,T]
    cost_temp += Terminal_Cost(xx_temp[:,TT], xx_ref[:,TT], params)['cost_T']

    # Cost structure collecting the cost registered for each gamma (step size)
    armijo_cost = np.concatenate(armijo_cost, cost_temp, axis=1)

    descent = 1 #!! solo per oggi
    if (armijo_cost[-1] <= (cost_kk) + cc*gammas[-1]*descent):  #descent va cambiato!!
      return gammas
    
    # Stucture collecting all the gamma computed not satisfying the Armijo's condition
    gammas = np.concatenate(gammas, beta*gammas[-1],axis=1)
    

  # OUTPUT: 
  #   - gammas : array containing all the step sizes computed by the Armijo's 
  #              Loop, gamma[-1] (the last element of the array) is the valid 
  #              step sized should be considered
  

**TRAJECTORY UPDATE** (Trajectory Update Function)

In [ ]:
def Trajectory_Update(kk, xx, uu, xx_ref, uu_ref, xx_init, TT, cost_kk, gamma, Sigma, KK, pp,  params):
  # INPUTs:
  #   - kk       : actual iteration
  #   - xx       : system state tensor 
  #   - uu       : input tensor
  #   - xx_ref   : referance state vector
  #   - uu_ref   : reference input vector
  #   - xx_init  : system initial state at at time t=0 
  #   - TT       : final step T 
  #   - cost_kk  : cost at k-th iteration
  #   - gamma    : Step size
  #   - Sigma    : Control Affine Element from the DDP algorithm
  #   - KK       : Feedback Gain Matrix frm the DDP algorithm
  #   - pp       : vector p from the DDP algorithm 
  #   - params   : parameter dictionary

  

**TRAJECTORY EXPLORATION** (Task 1)

In [6]:
# PARAMETERS SETTING

max_iterations = 200;

nx = 4; # nmumber of states 
nu = 1; # number of inputs
tf = 50; # Seconds

params = {'dt':0.001,   # Step size - Forward Euler method
          'gg':9.81,    # gravitational acceleration [m/s^2]
          'mm':0.12,    # ball mass [kg]
          'rr':0.015,   # ball radius [m]
          'ii':11.6E-6, # ball inertia [kg*m^2]
          'II':0.1125,  # beam inertia [kg*m^2]
          'LL':1        # beam lenght [m]
          'QQ':np.array([[100, 0, 0 , 0],
                              [ 0 ,10, 0 , 0],
                              [ 0 , 0,100, 0],
                              [ 0 , 0, 0 ,10]]),
          'RR':1,
          'QQ_T':np.array([[100, 0, 0 , 0],
                                [ 0 ,10, 0 , 0],
                                [ 0 , 0,100, 0],
                                [ 0 , 0, 0 ,10]])
              }

TT = int(tf/sys_params['dt']); # rounding down of the time span divided in time in number oof time intervals

xx_init = np.zeros((nx,1)); # initial state 



**Reference Definition** \
Step reference change between two equilibria 

In [7]:
# REFERENCE DEFINITION

ref_pos = 0.8; # ball reference position on the beam [m]
ref_inp = sys_params['mm']*sys_params['gg']*ref_pos; # input reference torque at reference pos ref_pos

# state reference definition:
xx_ref = np.zeros((nx,TT));
xx_ref[0,round(TT/2):TT] = ref_pos;

# input reference definition
uu_ref = np.zeros((nu,TT));
uu_ref[0,round(TT/2):TT] = ref_inp;

xx = np.zeros((nx,TT,max_iterations)); # state tensor
uu = np.zeros((nu,TT,max_iterations)); # input tensor

JJ      = np.zeros((max_iterations,1)); # cost function values memory
descent = np.zeros((max_iterations,1)); 



**TRAJECTORY OPTIMIZATION** (Task 2)

In [43]:
a = np.array([[1,2,3]])
B = np.array([[1,2,3],[4,4,6],[7,8,9]])
print(np.linalg.inv(B))
print(np.linalg.inv(B).T)


[[-1.00000000e+00  5.00000000e-01  1.26882631e-16]
 [ 5.00000000e-01 -1.00000000e+00  5.00000000e-01]
 [ 3.33333333e-01  5.00000000e-01 -3.33333333e-01]]
[[-1.00000000e+00  5.00000000e-01  3.33333333e-01]
 [ 5.00000000e-01 -1.00000000e+00  5.00000000e-01]
 [ 1.26882631e-16  5.00000000e-01 -3.33333333e-01]]


**TRAJECTORY TRACKING** (Task 3)